# Testing the Water Data for submission into 2023 Spring Water submission
This is the second notebook in creating a dependable model for validating the related water data, and do the tests on Nebraska Water system. 

In [1]:
# Import the libraries for water data validation
import pandas as pd
from libraries import general, Validators_2023_Water as vw

## Inventory Data
Checking the dataset and creating validation report

In [2]:
inventory = pd.read_excel('/Users/babak.jfard/projects/ETHTracking/Data/Water_Data/PWSInventory.xlsx')
#inventory = pd.read_excel('/Users/babak.jfard/projects/ETHTracking/Data/Water_Data/PWSInventory_latest.xlsx')

In [3]:
inventory.columns.tolist()

['StateFIPSCode',
 'PWSIDNumber',
 'YearAssociatedTo',
 'YearPulled',
 'PWSName',
 'PrincipalCountyServedName',
 'PrincipalCountyServed FIPS',
 'PrincipalCityName',
 '\ufeffPrincipalCityFeatureId',
 'TotalConnections',
 'SystemPopulation',
 'PrimarySourceCode',
 'FirstOfHoriz_Ref_Datum',
 'MinOfLatitude',
 'MinOfLongitude',
 'LocationDerivationCode',
 'LocationFlag']

In [4]:
inventory.columns = inventory.columns.str.replace('\ufeff', '')

#Change the names of several columns to match the names in the validator
inventory.rename(columns={'PrincipalCountyServed FIPS': 'PrincipalCountyServedFIPS', '\ufeffPrincipalCityFeatureId': 'PrincipalCityFeatureID'}, inplace=True)

In [5]:
# Adding a unique identifier for each row
inventory['RowIdentifier'] = inventory.index

In [6]:
counties = general.get_Counties_FIPS(state='NE')
# counties = counties['fips'].to_list()

### Validating
We will use PWS_Inventory class from vw to validate the data in inventory. Each row will be turned into a PWS_Inventory class

In [7]:
# Validating each row of inventory dataframe against the PWS_Inventory class
# Creat a list that will contain the RowIdentifier of the valid rows
from pydantic import ValidationError

valid_rows = []
# Creat a dictionary that contains the RowIdentifier of the invalid rows and the error message
invalid_rows = {}
for index, row in inventory.iterrows():
    
    try:
        vw.PWS_Inventory(counties, **row)
        # If passeed, add RowIdentifier into valid_rows list
        valid_rows.append(row['RowIdentifier'])

    except ValidationError as e:
        # If failed, add RowIdentifier and the error message into invalid_rows dictionary
        invalid_rows[row['RowIdentifier']] = e

        print(e)

ValueError: "PWS_Inventory" object has no field "counties"

In [31]:
inventory.columns

Index(['StateFIPSCode', 'PWSIDNumber', 'YearAssociatedTo', 'YearPulled',
       'PWSName', 'PrincipalCountyServedName', 'PrincipalCountyServedFIPS',
       'PrincipalCityName', 'PrincipalCityFeatureId', 'TotalConnections',
       'SystemPopulation', 'PrimarySourceCode', 'Horiz_Ref_Datum', 'Latitude',
       'Longitude', 'LocationDerivationCode', 'RowIdentifier'],
      dtype='object')

In [38]:
# get a list of the field keys of PWS_Inventory class


['RowIdentifier', 'PWSIDNumber', 'YearAssociatedTo', 'YearPulled', 'PWSName', 'PrincipalCountyServedFIPS', 'PrincipalCityFeatureID', 'TotalConnections', 'SystemPopulation', 'PrimarySourceCode', 'Latitude', 'Longitude', 'LocationDerivationCode']
